In [3]:
import numpy as np
import pickle

import json
import pandas as pd
import geopandas as gpd
import argparse

import math
from shapely.geometry import Point, MultiPolygon, Polygon

import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
#from computeDistributions import *

from functools import wraps
from time import time

In [32]:
def set_geoDF(input_dir):
    #assert fileExists(Path(input_dir, inputfiles["citygeojson"])), f"{inputfiles['citygeojson']} missing"
    
    geoDF = gpd.read_file(Path(input_dir,"city.geojson"))
    #geoDF =pd.DataFrame(geoDF)

    necessary_cols = ['wardNo', 'wardName', 'geometry']
    for col in necessary_cols:
        assert col in geoDF.columns
    geoDF = geoDF[necessary_cols]

    # read this for Polygon to MultiPolygon: https://gis.stackexchange.com/questions/311320/casting-geometry-to-multi-using-geopandas
    geoDF["geometry"] = [MultiPolygon([feature]) if isinstance(feature, Polygon) else feature for feature in geoDF["geometry"]]

    geoDF['wardBounds'] = geoDF.apply( (lambda row:  MultiPolygon(row['geometry']).bounds), axis=1)
    
    geoDF['wardCentre'] = geoDF.apply(
        lambda row: (
            MultiPolygon(row['geometry']).centroid.x, 
            MultiPolygon(row['geometry']).centroid.y
        ),
        axis=1
        )
    # geoDF['wardCentre'] = geoDF.apply(
    #     lambda row: (
    #         row['MultiPolygon'].centroid.x, 
    #         row['MultiPolygon'].centroid.y
    #     ),
    #     axis=1
    #     )    
    geoDF['wardNo'] = geoDF['wardNo'].astype(int)

    return geoDF

In [37]:
gdf=set_geoDF('./data/base/hillsborough_fl')
#gdf=set_geoDF('./data/base/nyc')

In [38]:
gdf

,wardNo,wardName,geometry,wardBounds,wardCentre
0,44,33620,"MULTIPOLYGON (((-82.42300 28.05705, -82.41988 ...","(-82.423, 28.054439, -82.402704, 28.06863)","(-82.41171525243381, 28.06112097623043)"
1,7,33548,"MULTIPOLYGON (((-82.46510 28.10038, -82.46352 ...","(-82.516864, 28.09513, -82.460102, 28.184577)","(-82.48183504472497, 28.1378796210974)"
2,11,33559,"MULTIPOLYGON (((-82.43878 28.10396, -82.43075 ...","(-82.438775, 28.079805, -82.369857, 28.229404)","(-82.41006546500103, 28.15787766549679)"
3,1,33503,"MULTIPOLYGON (((-82.28148 27.77024, -82.27475 ...","(-82.281531, 27.759132, -82.261502, 27.770308)","(-82.27424988880894, 27.763751772293507)"
4,28,33603,"MULTIPOLYGON (((-82.48665 27.98689, -82.48459 ...","(-82.486759, 27.969958, -82.442933, 28.001443)","(-82.46458051842482, 27.98526155659967)"
5,25,33596,"MULTIPOLYGON (((-82.26012 27.91803, -82.25999 ...","(-82.269292, 27.857304, -82.188926, 27.918046)","(-82.23591088921542, 27.887653115170636)"
6,40,33616,"MULTIPOLYGON (((-82.55483 27.85151, -82.55182 ...","(-82.554831, 27.840787, -82.506356, 27.889469)","(-82.52869677855865, 27.867122609211364)"
7,5,33534,"MULTIPOLYGON (((-82.41742 27.79826, -82.41252 ...","(-82.417715, 27.791551, -82.344669, 27.859302)","(-82.37505194509387, 27.822734166439393)"
8,22,33584,"MULTIPOLYGON (((-82.34387 28.03664, -82.33540 ...","(-82.343875, 27.959654, -82.244341, 28.042531)","(-82.28959010786855, 28.00637283269495)"
9,6,33547,"MULTIPOLYGON (((-82.26929 27.85723, -82.26929 ...","(-82.2693, 27.646229, -82.021214, 27.907726)","(-82.13665448940509, 27.772391548864963)"
